In [1]:
from collatex import *
import graphviz
import os
import xml.etree.ElementTree as ET
import re
inputfile="input/Sammeltranskripte.xml"
txtpath="txt"

Options to normalize text and generate Dictionary with plain text

In [2]:
def clean_text(text):
    # Remove leading and trailing whitespaces
    text = text.strip()
    # Remove double whitespaces
    text = " ".join(text.split())
    # Remove linebreaks
    text = text.replace("\n", "")
    # to lowercase
    text = text.lower()
    text = re.sub(r"[\s+]", " ", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = text.strip() 
    return text
    
def input_to_dict(file):
    # Parse the XML file
    tree = ET.parse(file)
    root = tree.getroot()
    # Create a dictionary to store the extracted text
    transcript_dict = {}
    # Find all <div> elements with attribute type="transcript"
    """ div_elements = root.findall('.//div[@type="transcript"]') """
    ms = root.findall('.//div[@type="ms"]')
    for m in ms:
        ms_id = m.attrib['corresp'].replace("-","")
        #print (ms_id)
        divs = m.findall('.//div[@type="transcript"]')
        alltranscripts=""
        for div in divs:
            alltranscripts += "".join(div.find('p').itertext())
            alltranscripts += " "
            # Get the ID of the transcript
            transcript_id = div.attrib['corresp'].replace("-","")
            #print(transcript_id)
            # Get the text of the transcript
            transcript_text = "".join(div.find('p').itertext())
            #print(transcript_text.replace('  ', '')) 
            # Add the text to the dictionary
            transcript_dict["_".join([transcript_id, ms_id])] = [transcript_id ,clean_text(transcript_text)]
        #transcript_dict[ms_id] = clean_text(alltranscripts)
    return transcript_dict
dict = input_to_dict(inputfile)


Save plaintext to .txt files in txt folder

In [3]:
def save_to_files(transcript_dict):
    # Save the transcript_dict into individual files
    for key, value in transcript_dict.items():
        #print(key, value[0])
        filename = f"{key}.txt"
        folder = txtpath+"/"+ value[0]+"/"
        
        mypath = ...
        if not os.path.isdir(folder):
           os.makedirs(folder)
        path =  folder +filename
        if value[1] != "" and value[1] != " ":
            with open(path, 'w') as file:
                file.write(value[1])
save_to_files(dict)

Start Collation

In [3]:
collation = Collation()

*Read files in 'txt' folder.*
Load content of each file as a witness into collateX with the filename as witness_name

In [4]:
directory = "txt/tadamnotm"

# Initialize a dictionary to store the file contents
file_contents = {}

# Iterate over each file in the directory
for filename in os.listdir(directory):
    #print(filename)
    if filename.endswith('.txt'):
        # Read the content of the file
        with open(os.path.join(directory, filename), 'r') as file:
            content = file.read()
            #print(content)
            # Use filename as variable name and store content as value
            file_contents[filename] = content

# Now you have a dictionary where keys are filenames and values are file contents
# You can also create variables for each file if needed
for filename, content in file_contents.items():
    #globals()[filename.split('.')[0]] = content
    #print(filename.split('.')[0])
    collation.add_plain_witness(filename.split('.')[0],content)


In [5]:
alignment_table = collate(collation, near_match=False, segmentation=False, layout="horizontal")
#print(alignment_table)

In [26]:
#collate(collation, output='svg_simple')

In [27]:
#collate(collation, output="html", layout="vertical")

In [6]:
alignment_csv = collate( collation, near_match=False, output="csv", segmentation=True )
with open( 'alignment_adam.csv', 'w' ) as csv_file:
    csv_file.write( alignment_csv )


## Next Step: Use cx2vartab.py and vartab2nex.py to convert .csv to .nex